In [154]:
import pandas as pd

In [155]:
test_data = pd.read_excel('./DATA.xlsx')

In [156]:
column_names = test_data.columns.tolist()
column_names

['Название рассылки',
 'Название кампании',
 'Направление',
 'Месяц',
 'Дата',
 'Год',
 'Номер недели',
 'День недели',
 'День недели.1',
 'Время',
 'Веб-версия',
 'Тема письма ',
 'Сегмент ',
 'Отправлено',
 'Доставлено',
 'Открытия',
 'Клики',
 'Баунсы (Все ошибки)',
 'Отписки',
 'UTM Метка',
 'Пользователей на сайте',
 'Воронка продаж. Шаг 1',
 'Воронка продаж. Шаг 2',
 'Воронка продаж. Шаг 3']

In [157]:
# лишние пробелы никто не любит : )

test_data = test_data.rename(columns={'Тема письма ': 'Тема письма'})

In [158]:
test_data.head(2)

,Название рассылки,Название кампании,Направление,Месяц,Дата,Год,Номер недели,День недели,День недели.1,Время,...,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,UTM Метка,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3
0,Название рассылки 1,Название кампании 1,Email,Октябрь,2021-10-27,2021,43,3,03-среда,19:24:00,...,741749.55,148349.91,17801.9892,39039.45,7417.4955,Метка 1,16377.830064,6336.582452,6209.850803,5154.176166
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,2021-11-05,2021,45,5,05-пятница,12:02:00,...,683401.50,123012.27,11071.1043,35968.50,6834.0150,Метка 10,10296.126999,3558.341491,3095.757097,2538.520820


In [159]:
grouped_months = test_data.groupby(['Месяц']).sum(['Доставлено','Отправлено','Открытия','Клики', 'Отписки'])
grouped_wd = test_data.groupby(['День недели.1']).sum(['Доставлено','Отправлено','Открытия','Клики', 'Отписки'])
grouped_by_theme = test_data.groupby(['Тема письма']).sum(['Доставлено','Отправлено','Открытия','Клики', 'Отписки'])

In [160]:
# add metrics to grouped frames

grouped_months['DR'] = round(grouped_months['Доставлено'] / grouped_months['Отправлено'] * 100, 2)
grouped_months['OR'] = round(grouped_months['Открытия'] / grouped_months['Доставлено'] * 100, 2)
grouped_months['CTOR'] = round(grouped_months['Клики'] / grouped_months['Открытия'] * 100, 2)
grouped_months['UR'] = round(grouped_months['Отписки'] / grouped_months['Доставлено'] * 100, 2)

grouped_wd['DR'] = round(grouped_wd['Доставлено'] / grouped_wd['Отправлено'] * 100, 2)
grouped_wd['OR'] = round(grouped_wd['Открытия'] / grouped_wd['Доставлено'] * 100, 2)
grouped_wd['CTOR'] = round(grouped_wd['Клики'] / grouped_wd['Открытия'] * 100, 2)
grouped_wd['UR'] = round(grouped_wd['Отписки'] / grouped_wd['Доставлено'] * 100, 2)

grouped_by_theme['DR'] = round(grouped_by_theme['Доставлено'] / grouped_by_theme['Отправлено'] * 100, 2)
grouped_by_theme['OR'] = round(grouped_by_theme['Открытия'] / grouped_by_theme['Доставлено'] * 100, 2)
grouped_by_theme['CTOR'] = round(grouped_by_theme['Клики'] / grouped_by_theme['Открытия'] * 100, 2)
grouped_by_theme['UR'] = round(grouped_by_theme['Отписки'] / grouped_by_theme['Доставлено'] * 100, 2)


In [161]:
# order by months 

new_order  = ["Январь", "Февраль", "Март", "Апрель", "Май", "Июнь", 
          "Июль", "Август", "Сентябрь", "Октябрь", "Ноябрь", "Декабрь"]
grouped_months = grouped_months.reindex(new_order, axis=0)

In [162]:
# main metrics by months

grouped_months[['DR','OR', 'CTOR', 'UR']]

,DR,OR,CTOR,UR
Месяц,,,,
Январь,98.14,12.98,8.24,4.02
Февраль,98.15,12.45,8.04,0.90
Март,98.14,13.78,7.71,0.90
Апрель,95.00,15.44,8.29,0.84
Май,95.48,14.89,9.40,0.66
Июнь,98.11,15.30,7.56,0.62
Июль,98.08,14.84,7.44,0.60
Август,98.18,15.27,7.33,0.56
Сентябрь,97.48,12.00,8.09,0.56


In [174]:
# best theme
# Как мне подсказывает логика, лучшей должна считаться тема с максимальным CTOR рейтингом, но при этом с минимальным количествоми отписок (что в приоиритете)
# Потому победителем в этой номинации становится.... :

best_theme = grouped_by_theme.sort_values(['UR', 'CTOR'], ascending=[True, False])
best_theme.drop(['Год', 'Номер недели', 'День недели'], axis = 1).head(1)

,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3,DR,OR,CTOR,UR
Тема письма,,,,,,,,,,,,,,
Тема письма 211,1953295,1914229.1,176109.0772,21133.089264,39065.9,2871.34365,17963.125874,10883.857967,10666.180808,8746.268263,98.0,9.2,12.0,0.15


In [175]:
# best day to mail
# Кажется, с днем чуть проще, потому просто сортируем по OR в разрезе дней недели

best_mail_day = grouped_wd.sort_values(['OR'], ascending=[False])
best_mail_day.drop(['Год', 'Номер недели', 'День недели'], axis = 1).head(1)

,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3,DR,OR,CTOR,UR
День недели.1,,,,,,,,,,,,,,
03-среда,54324966,5.301952e+07,7.573699e+06,635234.971428,1305450.225,660854.93322,560615.925858,221265.865662,188277.536531,149104.283096,97.6,14.28,8.39,1.25
